In [20]:
# Code for accessing Ollama and posing questions
# Test wether it works to invoke a model and pose a question

MODEL = "gemma:2b"
# MODEL = "mistral"
# MODEL = "llama2"


from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# Actual method to invoke a model and ask a question:
# model.invoke("Tell me a joke")

# Get Documents to apply the RAG to
Get a list of paths of docx documents I want to load into memory

In [1]:
import os

data_path = os.getenv("DOCUMENTS_PATH")

# Retrieve the path names of files in the specified directory
docx_file_paths = [os.path.join(data_path, file) for file in os.listdir(data_path)]
docx_file_paths_cleaned = [item for item in docx_file_paths if item.endswith('.docx')]

## Load in files to apply RAG to
Load the docx files into memory using the docx-python library

In [2]:
from langchain_community.document_loaders import Docx2txtLoader

loaded_texts = [Docx2txtLoader(path) for path in docx_file_paths_cleaned]
test_pages = loaded_texts[5].load_and_split()


## Create Template


In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")
print(prompt.format(context="Here is some context", question="Here is a question"))


## Create embeddings

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(test_pages, embedding = embeddings)

## Create vector store
For now it's in memory, but goals is to write it to disk (chromaDB?)

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("Initiatiefnaam")


# Create Chain

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
)


# Run the code

In [ ]:

#chain.invoke(
#    {
#
#    "question": "Geef de code van dit initiatief, deze heeft de vorm van AAA-11-11 " 
#    }
#)